In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
import datetime as datetime

#### Задание 1
##### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

##### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [2]:
def get_full_news(link_list):
    list_news_final = list()
    for i in link_list:
        time.sleep(0.3)
        res = requests.get(i)
        soup = BeautifulSoup(res.text, 'html.parser')
        text = soup.find('article', class_ = 'tm-page-article__content')

        final_text = text.find('div', class_ = 'article-formatted-body').text
            
        list_news_final.append(final_text)
    return(list_news_final)

def get_news(url, queries):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_ = 'tm-articles-list__item')
    articles_intro = list(map(lambda x: x.find('div', class_='tm-article-snippet'), posts))
    articles_intro_2 = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title'), articles_intro))
    a_list = list(map(lambda x: x.find('span'), articles_intro_2))
    list_news = list(map(lambda x: str(x)[6:-7], a_list))
    time_posts = soup.find_all('span', class_ = 'tm-article-snippet__datetime-published')
    time_list = list(map(lambda x: x.find('time').get('title'), time_posts))
    link_list = list(map(lambda x: 'https://www.habr.com' + x.find('a').get('href'), articles_intro_2))
    list_news_final = get_full_news(link_list)
    news_df = pd.concat([pd.Series(time_list, name='Дата'), pd.Series(list_news, name='Заголовок'), pd.Series(link_list,name='Ссылка'), pd.Series(list_news_final,name='Текст_статьи')], axis = 1)
    for i in list_news:
        for k in queries:
            if k.lower() in i.lower():
                list_news_final.append(i)
    final_df = pd.DataFrame()
    for i in list_news_final:
        final_df = pd.concat([final_df, news_df[news_df['Заголовок'] == i]])
    return(final_df)

In [3]:
KEYWORDS = ['python', 'парсинг', 'как']
url = 'https://habr.com/ru/all/'

In [4]:
get_news(url, queries=KEYWORDS)

,Дата,Заголовок,Ссылка,Текст_статьи
3,"2021-08-23, 22:17",Вакцин от COVID-19 недостаточно — необходимо п...,https://www.habr.com/ru/post/574298/,Самое мощное оружие против COVID — это не лека...
4,"2021-08-23, 22:04",Как подружить ClickHouse и Power Bi,https://www.habr.com/ru/post/574294/,Однажды на работе мне потребовалось построить ...
7,"2021-08-23, 19:01",Бывший проектировщик микросхем для Военно-Возд...,https://www.habr.com/ru/post/574276/,Если вы изучили начала цифровой схемотехники и...
17,"2021-08-23, 15:38",Как автоматизировать сбор статистики из Яндекс...,https://www.habr.com/ru/post/574224/,"У Яндекс.Дзен нет готового API, чтобы агрегиро..."
19,"2021-08-23, 15:13",Кранчи в IT или Как убивает погоня за сроками,https://www.habr.com/ru/company/skillfactory/b...,"Многие игроки считают, что разработка — это ин..."


#### Задание 2.
##### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

##### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology’
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

#### обязательная часть

In [9]:
def avast_func(url, headers, email):
    email_dict = dict.fromkeys(['emailAddresses'], email)
    avast_res = requests.post(URL, headers=HEADERS, json=email_dict)

    avast_df = pd.DataFrame(columns=['email','site', 'descripriton','publishDate'])
    for i in avast_res.json()['summary']:
        for j in avast_res.json()['summary'][i].values():
            for key in j:
                avast_df.loc[len(avast_df)] = [str(i), \
                                           avast_res.json()['breaches'][str(key)]['site'], \
                                           avast_res.json()['breaches'][str(key)]['description'], \
                                           avast_res.json()['breaches'][str(key)]['publishDate']]
    return(avast_df)

In [24]:
EMAIL = ['xxx@x.ru', 'yyy@y.ru', 'lolkek@cheburek.ru']

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

HEADERS = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
    'Connection': 'keep-alive',
    'Content-Length': '32',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

In [25]:
avast_func(url=URL, email=EMAIL, headers=HEADERS)

,email,site,descripriton,publishDate
0,xxx@x.ru,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z
1,xxx@x.ru,vk.com,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z
2,xxx@x.ru,imesh.com,"In June 2016, a cache of over 51 million user ...",2016-10-23T00:00:00Z
3,xxx@x.ru,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da...",2017-01-31T00:00:00Z
4,xxx@x.ru,cfire.mail.ru,"In July and August of 2016, two criminals carr...",2017-02-14T00:00:00Z
5,xxx@x.ru,parapa.mail.ru,"In July and August 2016, two criminals execute...",2017-02-14T00:00:00Z
6,yyy@y.ru,vk.com,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z
7,yyy@y.ru,qip.ru,"In 2011, Russian instant messaging service pro...",2016-11-01T00:00:00Z


#### Дополнительная часть

In [15]:
def vk_group_news(NEWSFEED_REQUEST, params):
    res = requests.get(NEWSFEED_REQUEST, params)
    wall_group = pd.DataFrame(res.json()['response']['items'])[['text', 'date']]
    wall_group['normal_date'] = wall_group['date'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    return(wall_group)

In [20]:
# https://vk.com/dev/manuals
# https://vk.com/dev/wall.get
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
with open('token.txt', 'r') as obj:
    TOKEN = obj.read().strip()
VERSION = '5.103'
SLEEP = 0.33

In [22]:
# обращаем внимание, что максимальное количество постов, 
# которые можно вытащить за раз, ограничено
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': 'netology',
    'count': 50
}

In [23]:
vk_group_news(NEWSFEED_REQUEST=NEWSFEED_REQUEST, params=params)

,text,date,normal_date
0,Наш летний диджитал-марафон продолжается 🎉 \n ...,1629712808,2021-08-23 10:00:08
1,Обучение на неподходящем курсе может обернутьс...,1629731280,2021-08-23 15:08:00
2,Дизайн — это не только про картинки 🤔 \n \nВме...,1629706500,2021-08-23 08:15:00
3,*партнёрский пост* \n \nВы когда-нибудь задумы...,1629644820,2021-08-22 15:07:00
4,Смена профессии и поиск новой работы — волните...,1629621540,2021-08-22 08:39:00
5,"Рассказываем, где была Нетология ☺️\n\nПублика...",1629558007,2021-08-21 15:00:07
6,Онлайн-обучение позволяет следовать принципам ...,1629535320,2021-08-21 08:42:00
7,"До конца лета осталось совсем немного, пора по...",1629468001,2021-08-20 14:00:01
8,"Заботливо собрали подборку бесплатных занятий,...",1629446340,2021-08-20 07:59:00
9,"Когда телефон лежит где-то рядом — кажется, чт...",1629389220,2021-08-19 16:07:00


In [26]:
del TOKEN

### понимаю, что нужно оставлять комменты, но отстаю дико поэтому погнал дальше))